## 1. import Libs

In [1]:
import torch
from torch.autograd import Variable
from torchvision import transforms
from dataset import DatasetFromFolder
from model import Generator, Discriminator

import utils
import argparse
import os, itertools
from logger import Logger
import numpy as np

/home/ncp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2. Setting Hyperparameters

In [2]:
parser = argparse.ArgumentParser()

#Data Set Parameter
parser.add_argument('--dataset', required=False, default='horse2zebra', help='input dataset')
parser.add_argument('--batch_size', type=int, default=1, help='train batch size')
parser.add_argument('--input_size', type=int, default=256, help='input size')
parser.add_argument('--resize_scale', type=int, default=286, help='resize scale (0 is false)')
parser.add_argument('--crop_size', type=int, default=256, help='crop size (0 is false)')
parser.add_argument('--fliplr', type=bool, default=True, help='random fliplr True of False')

#Model Parameters 
parser.add_argument('--ngf', type=int, default=32) # number of generator filters
parser.add_argument('--ndf', type=int, default=64) # number of discriminator filters
parser.add_argument('--num_resnet', type=int, default=6, help='number of resnet blocks in generator')

#Learning Parameters
parser.add_argument('--num_epochs', type=int, default=200, help='number of train epochs')
parser.add_argument('--decay_epoch', type=int, default=100, help='start decaying learning rate after this number')
parser.add_argument('--lrG', type=float, default=0.0002, help='learning rate for generator, default=0.0002')
parser.add_argument('--lrD', type=float, default=0.0002, help='learning rate for discriminator, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
parser.add_argument('--lambdaA', type=float, default=10, help='lambdaA for cycle loss')
parser.add_argument('--lambdaB', type=float, default=10, help='lambdaB for cycle loss')
params = parser.parse_args([])
print(params)

# Directories for loading data and saving results
data_dir = 'data/' + params.dataset + '/'
save_dir = params.dataset + '_results/'
model_dir = params.dataset + '_model/'

Namespace(batch_size=1, beta1=0.5, beta2=0.999, crop_size=256, dataset='horse2zebra', decay_epoch=100, fliplr=True, input_size=256, lambdaA=10, lambdaB=10, lrD=0.0002, lrG=0.0002, ndf=64, ngf=32, num_epochs=200, num_resnet=6, resize_scale=286)


## 3. Making directories

In [3]:
if not os.path.exists(data_dir) : 
    os.makedirs(data_dir)
    file = 'ZIP_FILE.zip'
    url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/' + params.dataset + '.zip'
    print(url)
    !wget -N $url -O $file
    !unzip $file -d 'data/'
    !rm $file
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# Set the logger
D_A_log_dir = save_dir + 'D_A_logs'
D_B_log_dir = save_dir + 'D_B_logs'
if not os.path.exists(D_A_log_dir):
    os.mkdir(D_A_log_dir)
if not os.path.exists(D_B_log_dir):
    os.mkdir(D_B_log_dir)

G_A_log_dir = save_dir + 'G_A_logs'
G_B_log_dir = save_dir + 'G_B_logs'
if not os.path.exists(G_A_log_dir):
    os.mkdir(G_A_log_dir)
if not os.path.exists(G_B_log_dir):
    os.mkdir(G_B_log_dir)

cycle_A_log_dir = save_dir + 'cycle_A_logs'
cycle_B_log_dir = save_dir + 'cycle_B_logs'
if not os.path.exists(cycle_A_log_dir):
    os.mkdir(cycle_A_log_dir)
if not os.path.exists(cycle_B_log_dir):
    os.mkdir(cycle_B_log_dir)

img_log_dir = save_dir + 'img_logs'
if not os.path.exists(img_log_dir):
    os.mkdir(img_log_dir)

https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
for details.

--2018-06-26 21:40:41--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116867962 (111M) [application/zip]
Saving to: ‘ZIP_FILE.zip’

ZIP_FILE.zip        100%[===================>] 111.45M  1.27MB/s    in 1m 55s  

2018-06-26 21:42:38 (992 KB/s) - ‘ZIP_FILE.zip’ saved [116867962/116867962]

Archive:  ZIP_FILE.zip
   creating: data/horse2zebra/trainA/
  inflating: data/horse2zebra/trainA/n02381460_6223.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1567.jpg  
  inflating: data/horse2zebra/trainA/n02381460_3354.jpg  
  inflating: data/horse2zebra/trainA/n02381460_299.jpg  
  inflating: data/horse2zebra/trainA/n02381460_3001.jp

  inflating: data/horse2zebra/trainA/n02381460_3066.jpg  
  inflating: data/horse2zebra/trainA/n02381460_5442.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1796.jpg  
  inflating: data/horse2zebra/trainA/n02381460_643.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1487.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1025.jpg  
  inflating: data/horse2zebra/trainA/n02381460_5618.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2578.jpg  
  inflating: data/horse2zebra/trainA/n02381460_8306.jpg  
  inflating: data/horse2zebra/trainA/n02381460_105.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4748.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4944.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1952.jpg  
  inflating: data/horse2zebra/trainA/n02381460_5721.jpg  
  inflating: data/horse2zebra/trainA/n02381460_919.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2779.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1422.jpg  
  inflating: data

  inflating: data/horse2zebra/trainA/n02381460_2339.jpg  
  inflating: data/horse2zebra/trainA/n02381460_6466.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2093.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1651.jpg  
  inflating: data/horse2zebra/trainA/n02381460_5704.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7412.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2936.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4917.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2349.jpg  
  inflating: data/horse2zebra/trainA/n02381460_3694.jpg  
  inflating: data/horse2zebra/trainA/n02381460_36.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4779.jpg  
  inflating: data/horse2zebra/trainA/n02381460_3942.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1856.jpg  
  inflating: data/horse2zebra/trainA/n02381460_61.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1614.jpg  
  inflating: data/horse2zebra/trainA/n02381460_6782.jpg  
  inflating: data/

  inflating: data/horse2zebra/trainA/n02381460_4608.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1703.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1266.jpg  
  inflating: data/horse2zebra/trainA/n02381460_3746.jpg  
  inflating: data/horse2zebra/trainA/n02381460_971.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4125.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7493.jpg  
  inflating: data/horse2zebra/trainA/n02381460_4538.jpg  
  inflating: data/horse2zebra/trainA/n02381460_8088.jpg  
  inflating: data/horse2zebra/trainA/n02381460_6978.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2217.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1736.jpg  
  inflating: data/horse2zebra/trainA/n02381460_394.jpg  
  inflating: data/horse2zebra/trainA/n02381460_757.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7348.jpg  
  inflating: data/horse2zebra/trainA/n02381460_935.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1711.jpg  
  inflating: data/

  inflating: data/horse2zebra/trainA/n02381460_3732.jpg  
  inflating: data/horse2zebra/trainA/n02381460_419.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7622.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7178.jpg  
  inflating: data/horse2zebra/trainA/n02381460_8586.jpg  
  inflating: data/horse2zebra/trainA/n02381460_6761.jpg  
  inflating: data/horse2zebra/trainA/n02381460_588.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1909.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1957.jpg  
  inflating: data/horse2zebra/trainA/n02381460_2056.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1074.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1418.jpg  
  inflating: data/horse2zebra/trainA/n02381460_451.jpg  
  inflating: data/horse2zebra/trainA/n02381460_7692.jpg  
  inflating: data/horse2zebra/trainA/n02381460_6297.jpg  
  inflating: data/horse2zebra/trainA/n02381460_764.jpg  
  inflating: data/horse2zebra/trainA/n02381460_1722.jpg  
  inflating: data/

...

  inflating: data/horse2zebra/trainB/n02391049_2136.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2057.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3905.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2104.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1946.jpg  
  inflating: data/horse2zebra/trainB/n02391049_533.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5909.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2034.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6053.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3276.jpg  
  inflating: data/horse2zebra/trainB/n02391049_909.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6951.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2177.jpg  
  inflating: data/horse2zebra/trainB/n02391049_10158.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6386.jpg  
  inflating: data/horse2zebra/trainB/n02391049_614.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5189.jpg  
  inflating: dat

  inflating: data/horse2zebra/trainB/n02391049_9062.jpg  
  inflating: data/horse2zebra/trainB/n02391049_121.jpg  
  inflating: data/horse2zebra/trainB/n02391049_10966.jpg  
  inflating: data/horse2zebra/trainB/n02391049_314.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6957.jpg  
  inflating: data/horse2zebra/trainB/n02391049_659.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3365.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6564.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9365.jpg  
  inflating: data/horse2zebra/trainB/n02391049_788.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5208.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2637.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5985.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2671.jpg  
  inflating: data/horse2zebra/trainB/n02391049_4.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2922.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1992.jpg  
  inflating: data/ho

  inflating: data/horse2zebra/trainB/n02391049_2879.jpg  
  inflating: data/horse2zebra/trainB/n02391049_7457.jpg  
  inflating: data/horse2zebra/trainB/n02391049_63.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9979.jpg  
  inflating: data/horse2zebra/trainB/n02391049_407.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5169.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2593.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2003.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3251.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3333.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6622.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3771.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2964.jpg  
  inflating: data/horse2zebra/trainB/n02391049_8982.jpg  
  inflating: data/horse2zebra/trainB/n02391049_8021.jpg  
  inflating: data/horse2zebra/trainB/n02391049_4965.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3326.jpg  
  inflating: data

  inflating: data/horse2zebra/trainB/n02391049_10754.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1355.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6317.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3319.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3805.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9625.jpg  
  inflating: data/horse2zebra/trainB/n02391049_587.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2518.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9951.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9317.jpg  
  inflating: data/horse2zebra/trainB/n02391049_713.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2322.jpg  
  inflating: data/horse2zebra/trainB/n02391049_127.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6856.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1378.jpg  
  inflating: data/horse2zebra/trainB/n02391049_605.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3317.jpg  
  inflating: data

  inflating: data/horse2zebra/trainB/n02391049_8102.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3037.jpg  
  inflating: data/horse2zebra/trainB/n02391049_6512.jpg  
  inflating: data/horse2zebra/trainB/n02391049_3265.jpg  
  inflating: data/horse2zebra/trainB/n02391049_968.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5333.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1062.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2306.jpg  
  inflating: data/horse2zebra/trainB/n02391049_7678.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2826.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5015.jpg  
  inflating: data/horse2zebra/trainB/n02391049_7434.jpg  
  inflating: data/horse2zebra/trainB/n02391049_2225.jpg  
  inflating: data/horse2zebra/trainB/n02391049_506.jpg  
  inflating: data/horse2zebra/trainB/n02391049_5005.jpg  
  inflating: data/horse2zebra/trainB/n02391049_1198.jpg  
  inflating: data/horse2zebra/trainB/n02391049_9122.jpg  
  inflating: dat